In [1]:
import xml.etree.ElementTree as ET
import os

import pandas as pd
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
from tqdm.autonotebook import tqdm
from pandarallel import pandarallel

tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/Users/maddy/Documents/2021/markup/investigations-data-broker-lobbying/databrokers/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/maddy/Documents/2021/markup/investigations-data-broker-lobbying/databrokers/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


### Load Data Brokers

Import CA data brokers list

In [2]:
fn = '../data/data_brokers/ca-data-brokers.csv'
df = pd.read_csv(fn)
df['state'] = 'CA'
ca = df[['Data Broker Name', 'Email Address', 'Website URL', 'Physical Address', 'state']].copy()
ca.rename(inplace=True, columns={
    'Data Broker Name':'name',
    'Email Address':'email',
    'Website URL':'url',
    'Physical Address':'address'
})

Import VT data brokers list

In [3]:
fn = '../data/data_brokers/vt-data-brokers.csv'
df = pd.read_csv(fn)
df['state'] = "VT"

vt = df[['Data Broker Name:','Address:', 'Email Address:', 'Primary Internet Address:', 'state']].copy()
vt.rename(inplace=True, columns={
    'Data Broker Name:':'name',
    'Address:':'address',
    'Email Address:':'email',
    'Primary Internet Address:':'url'
})

Merge the two

In [4]:
brokers = pd.concat([ca, vt])

Save as output

In [5]:
brokers.to_csv('../data/matching_process/brokers.csv', index=False)

### Load Lobbyist Clients

In [6]:
client_list = []

folder = '../data/lobbying/'
for path, dirs, files in os.walk(folder):
    for file in files:
        fullpath = os.path.join(path, file)
        if file.endswith(".xml"):
            with open(fullpath, "rb") as data:
                tree = ET.parse(data)
                root = tree.getroot()
                for filing in root.iter('Filing'):
                    filing_info = filing.attrib
                    for client in filing.iter('Client'):
                        client_info = client.attrib
                        info = {
                            'filing.id': filing_info['ID'],
                            'filing.period': filing_info['Period'],
                            'filing.year': filing_info['Year'],
                            'client.name': client_info['ClientName'],
                            'client.id': client_info['ClientID'],
                            'client.desc': client_info['GeneralDescription'],
                            'client.state': client_info['ClientState'],
                            'client.country': client_info['ClientCountry']
                        }
                        client_list.append(info)

cf = pd.DataFrame(client_list)

Filter for just 2020 filings

In [7]:
clients = cf[cf['filing.year'] == '2020'].copy()

Add bridge to matches

In [8]:
clients['client.name.check'] = clients['client.name'].str.replace(",","").str.replace(".","").str.upper()

/Users/maddy/Documents/2021/markup/investigations-data-broker-lobbying/databrokers/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


Save as output

In [9]:
clients.to_csv('../data/matching_process/clients.csv', index=False)

### Guess Matches

In [10]:
brokers['name.check'] = brokers['name'].str.replace(",","").str.replace(".","").str.upper()

/Users/maddy/Documents/2021/markup/investigations-data-broker-lobbying/databrokers/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [11]:
unique_clients = pd.DataFrame()
unique_clients['client.name.check'] = clients['client.name.check'].unique()

choices = list(brokers['name.check'].unique())
choices.extend([
    'EQUIFAX',
    'EXPERIAN',
    'X-MODE',
    'IHS MARITIME & TRADE',
    'ACXIOM',
    'DELOITTE', 
    'PUBLICIS GROUP', 
    'ORACLE',
    'ACCENTURE FEDERAL SERVICES',
    'RELX',
    'ELSEVIER',
    'LIVERAMP',
    'INMAR',
    'EPSILON DATA'])

def guess(client):
    if client in choices:
        return client, 100
    pick, score = process.extractOne(client, choices)
    return pick, score

### First Pass: Data Broker Name List

In [ ]:
unique_clients['guess'] = unique_clients['client.name.check'].parallel_apply(guess)

In [ ]:
unique_clients[['guess.name', 'guess.confidence']] = unique_clients['guess'].apply(pd.Series)

### Export for Human Double-Checking

In [ ]:
describe = unique_clients['guess.confidence'].describe()

In [ ]:
guesses = unique_clients[unique_clients['guess.confidence'] > describe['75%']].sort_values(by='guess.confidence', ascending=False)
guesses.to_csv('../data/matching_process/match-guesses.csv', index=False)